In [1]:
import notebook_loader
from model import multiple_layer_model as mlp

importing notebook from /data1/dhyoon/tensor_work/book_main/model/multiple_layer_model.ipynb


In [2]:
mlp.random_fix = True

In [3]:
id1 = mlp.IrisDataset("regression")

m1 = mlp.MultiLayerPerceptronModel("iris-regression-test-hid-1", id1, [10])
m1.train(epoch_count=100)

m2 = mlp.NoTFMLPModel("notf-iris-regression-hid-1", id1, [10])
m2.train(epoch_count=100)

Model iris-regression-test-hid-1 train report:
    Epoch 10: cost=0.548, accuracy=0.070/-0.261 (1/1 secs)
    Epoch 20: cost=0.264, accuracy=-0.211/-0.654 (0/1 secs)
    Epoch 30: cost=0.229, accuracy=-0.134/-0.556 (0/1 secs)
    Epoch 40: cost=0.196, accuracy=-0.028/-0.441 (0/1 secs)
    Epoch 50: cost=0.165, accuracy=0.067/-0.314 (1/2 secs)
    Epoch 60: cost=0.137, accuracy=0.169/-0.178 (0/2 secs)
    Epoch 70: cost=0.112, accuracy=0.280/-0.052 (0/2 secs)
    Epoch 80: cost=0.092, accuracy=0.379/0.078 (0/2 secs)
    Epoch 90: cost=0.077, accuracy=0.472/0.194 (1/3 secs)
    Epoch 100: cost=0.066, accuracy=0.547/0.297 (0/3 secs)

Model notf-iris-regression-hid-1 train report:
    Epoch 10: cost=0.548, accuracy=-0.228/-0.642 (0/0 secs)
    Epoch 20: cost=0.264, accuracy=0.169/-0.162 (0/0 secs)
    Epoch 30: cost=0.229, accuracy=0.175/-0.090 (0/0 secs)
    Epoch 40: cost=0.196, accuracy=0.266/-0.014 (0/0 secs)
    Epoch 50: cost=0.165, accuracy=0.317/0.059 (0/0 secs)
    Epoch 60: cost=

In [4]:
print(babo)

NameError: name 'babo' is not defined

In [ ]:
import numpy as np
import tensorflow as tf
import time

In [ ]:
class SudokuDataset(mlp.Dataset):
    pass

def unpack_map(line):
    quiz = np.zeros([81], dtype=np.int8)
    solution = np.zeros([81], dtype=np.int8)
    for n in range(81):
        quiz[n] = int(line[n])
        solution[n] = int(line[n+82])
    return quiz, solution

def sudoku_init(self, max_count=0, train_ratio=0.80, valid_ratio=0.05):
    if mlp.random_fix: np.random.seed(3456)
    
    mlp.Dataset.__init__(self, "sudoku")
    
    quizzes, solutions = [], []

    for line in open("./data/sudoku.csv"):
        if line[0] == 'q': continue
        quiz, solution = unpack_map(line)
        
        
        open_cnt = np.random.randint(61-np.count_nonzero(quiz))
        
        for n in range(open_cnt):
            pos = np.random.randint(81)
            quiz[pos] = solution[pos]
            
        #quiz = solution
        #for n in range(5):
        #    pos = np.random.randint(81)
        #    quiz[pos] = 0
        #quizzes.append(solution)
        #solutions.append(solution)
        
        #open_cnt = int((81-np.count_nonzero(quiz)) / 10)
        #for n in range(5):
        #    quizzes.append(quiz)
        #    solutions.append(solution)
        #    quiz = fill_hints(quiz, solution, open_cnt)
            
        quizzes.append(quiz)
        solutions.append(solution)
        
        if max_count > 0 and len(quizzes) >= max_count: break

    ones = np.ones([81]).astype(int)
    solution_idxes = solutions - ones
    xs = np.asarray(quizzes)
    ys = np.eye(9)[solution_idxes].reshape(-1, 81*9)

    self.input_dim = 81
    self.output_dim = 81 * 9
    #self.output_dim = 81*9

    data_count = len(xs)
    train_count = int(data_count * train_ratio)
    valid_count = int(data_count * valid_ratio)
    test_count = data_count - train_count - valid_count
    print("sudoku data was prepared {}+{}+{}={} items".format(train_count, valid_count, test_count, data_count))

    test_start_idx = train_count + valid_count

    indices = np.arange(data_count)
    np.random.shuffle(indices)

    self.train_xs = xs[indices[0:train_count]]
    self.train_ys = ys[indices[0:train_count]]
    self.validate_xs = xs[indices[train_count:test_start_idx]]
    self.validate_ys = ys[indices[train_count:test_start_idx]]
    self.test_xs = xs[indices[test_start_idx:]]
    self.test_ys = ys[indices[test_start_idx:]]

    self.train_count = train_count
    
SudokuDataset.__init__ = sudoku_init

In [ ]:
class SudokuCnnModel(mlp.MultiLayerPerceptronModel):
    pass

def sudoku_init(self, name, dataset, hidden_dims, learning_rate=0.01):
    mlp.MultiLayerPerceptronModel.__init__(self, name, dataset, hidden_dims, learning_rate)

SudokuCnnModel.__init__ = sudoku_init

In [ ]:
def sudoku_train(self, epoch_count=10, batch_size=10):
    if batch_size == 0:
        batch_size = self.dataset.train_count
        
    batch_count = int(self.dataset.train_count / batch_size)
    report_period = epoch_count / 10
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    if mlp.random_fix: np.random.seed(1945)
    
    time1 = time2 = int(time.time())
    
    print("Model {} train report:".format(self.name))
    
    for epoch in range(epoch_count):
        costs, total_accs, blank_accs, max_accs =  [], [], [], []
        
        for n in range(batch_count):
            train_X, train_Y = self.dataset.get_train_data(batch_size, n)
            _, cost, total_acc, blank_acc, max_acc = \
                sess.run([self.train_op, self.loss, self.total_acc, self.blank_acc, self.max_acc], \
                         feed_dict={self.x:train_X, self.y:train_Y})
            costs.append(cost)
            total_accs.append(total_acc)
            blank_accs.append(blank_acc)
            max_accs.append(max_acc)
        
        if (epoch+1) % report_period == 0:
            validate_X, validate_Y = self.dataset.get_test_data(True, 30)
            total_acc, blank_acc, max_acc = sess.run([self.total_acc, self.blank_acc, self.max_acc], \
                                                          feed_dict={self.x:validate_X, self.y:validate_Y})

            time3 = int(time.time())
            print("    Epoch {}: cost={:5.3f}, accuracy={:5.3f}/{:5.3f}/{:5.3f}({:5.3f}/{:5.3f}/{:5.3f}) ({}/{} secs)". \
                  format(epoch+1, np.mean(costs), np.mean(total_accs), np.mean(blank_accs), np.mean(max_accs), total_acc, blank_acc, max_acc, time3-time2, time3-time1))
            time2 = time3

    """
    validate_X, validate_Y = self.dataset.get_test_data(True, 2)
    estimate, answer, correct, total_acc, non_blank_mask, non_blank_correct, non_blank_acc, \
    blank_mask, blank_correct, blank_acc, probs, max_probs, blank_probs, max_prob_pos, \
    max_prob_mask, max_prob_est, max_prob_ans, max_correct, max_acc = \
        sess.run([self.estimate, self.answer, self.correct, self.total_acc, \
                  self.non_blank_mask, self.non_blank_correct, self.non_blank_acc, \
                  self.blank_mask, self.blank_correct, self.blank_acc, \
                  self.probs, self.max_probs, self.blank_probs, self.max_prob_pos, self.max_prob_mask, \
                  self.max_prob_est, self.max_prob_ans, self.max_correct, self.max_acc], \
                                                    feed_dict={self.x:validate_X, self.y:validate_Y})
    print("estimate", np.reshape(estimate, [-1, 9]))
    print("answer", np.reshape(answer, [-1, 9]))
    print("correct", np.reshape(correct, [-1, 9]))
    print("total_acc", total_acc)
    print("x", np.reshape(validate_X, [-1, 9]))
    print("non_blank_mask", np.reshape(non_blank_mask, [-1, 9]))
    print("non_blank_correct", np.reshape(non_blank_correct, [-1, 9]))
    print("non_blank_acc", non_blank_acc)
    print("blank_mask", np.reshape(blank_mask, [-1, 9]))
    print("blank_correct", np.reshape(blank_correct, [-1, 9]))
    print("blank_acc", blank_acc)
    print("probs", np.reshape(probs, [-1, 9]))
    print("max_probs", np.reshape(max_probs, [-1, 9]))
    print("blank_probs", np.reshape(blank_probs, [-1, 9]))
    print("max_prob_pos", max_prob_pos)
    print("max_prob_mask", np.reshape(max_prob_mask, [-1, 9]))
    print("max_prob_est", max_prob_est)
    print("max_prob_ans", max_prob_ans)
    print("max_correct", max_correct)
    print("max_acc", max_acc)
    """
    
    path = "params/{}.ckpt".format(self.name)
    self.saver.save(sess, path)
    sess.close()

SudokuCnnModel.train = sudoku_train

In [ ]:
def sudoku_build_loss_accuracy(self):
    
    labels = tf.cast(tf.reshape(self.y, [-1, 81, 9]), "float")
    logits = tf.reshape(self.output, [-1, 81, 9])
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    loss = tf.reduce_mean(entropy)
    
    estimate = tf.argmax(logits, 2)
    answer = tf.argmax(labels, 2)
    correct = tf.cast(tf.equal(estimate, answer), "float")
    total_acc = tf.reduce_mean(correct)
    
    non_blank_mask = tf.cast(tf.sign(self.x), "float")
    non_blank_correct = non_blank_mask * correct
    non_blank_acc = tf.reduce_sum(non_blank_correct) / tf.reduce_sum(non_blank_mask)

    blank_mask = 1 - non_blank_mask
    blank_correct = blank_mask * correct
    blank_acc = tf.reduce_sum(blank_correct) / tf.reduce_sum(blank_mask)

    #self.estimate = estimate
    #self.answer = answer
    #self.correct = correct

    #self.non_blank_mask = non_blank_mask
    #self.non_blank_correct = non_blank_correct

    #self.blank_mask = blank_mask
    #self.blank_correct = blank_correct

    probs = tf.nn.softmax(logits)
    max_probs = tf.reduce_max(probs, axis=2)
    blank_probs = blank_mask * max_probs
    max_prob_pos = tf.argmax(blank_probs, 1)
    max_prob_mask = tf.one_hot(max_prob_pos, 81)
    max_prob_est = tf.reduce_sum(tf.cast(estimate, "float")*max_prob_mask, 1)
    max_prob_ans = tf.reduce_sum(tf.cast(answer, "float")*max_prob_mask, 1)
    max_correct = tf.cast(tf.equal(max_prob_est, max_prob_ans), "float")
    max_acc = tf.reduce_mean(max_correct)

    self.loss = loss
    #self.probs = probs
    #self.max_probs = max_probs
    #self.blank_probs = blank_probs
    #self.max_prob_pos = max_prob_pos
    #self.max_prob_mask = max_prob_mask
    #self.max_prob_est = max_prob_est
    #self.max_prob_ans = max_prob_ans
    #self.max_correct = max_correct
    self.total_acc = total_acc
    self.blank_acc = blank_acc
    self.max_acc = max_acc

SudokuCnnModel.build_loss_accuracy = sudoku_build_loss_accuracy

In [ ]:
sd1 = SudokuDataset(max_count=1000)

sm1 = SudokuCnnModel("sudoku-mlp-1", sd1, [256])
sm1.train(epoch_count=100)

In [ ]:
sd2 = SudokuDataset(max_count=10000)

sm2 = SudokuCnnModel("sudoku-mlp-2", sd2, [256])
sm2.train(epoch_count=10)

In [ ]:
sm3 = SudokuCnnModel("sudoku-mlp-3", sd1, [256])
sm3.train(epoch_count=1000)

In [ ]:
sm4 = SudokuCnnModel("sudoku-mlp-4", sd2, [256])
sm4.train(epoch_count=1000)